
# Get started with your own first training loop

**Author**: [Vincent Moens](https://github.com/vmoens)


<div class="alert alert-info"><h4>Note</h4><p>To run this tutorial in a notebook, add an installation cell
  at the beginning containing:

```
!pip install tensordict
!pip install torchrl</p></div>
```


In [15]:
pip install tensordict torchrl av

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


   ━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/39.9 MB 14.5 MB/s eta 0:00:03

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.8 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Time to wrap up everything we've learned so far in this Getting Started
series!

In this tutorial, we will be writing the most basic training loop there is
using only components we have presented in the previous lessons.

We'll be using DQN with a CartPole environment as a prototypical example.

We will be voluntarily keeping the verbosity to its minimum, only linking
each section to the related tutorial.

## Building the environment

We'll be using a gym environment with a :class:`~torchrl.envs.transforms.StepCounter`
transform. If you need a refresher, check our these features are presented in
`the environment tutorial <gs_env_ted>`.




In [2]:
import torch

torch.manual_seed(0)

import time

from torchrl.envs import GymEnv, StepCounter, TransformedEnv

env = TransformedEnv(GymEnv("CartPole-v1"), StepCounter())
env.set_seed(0)

from tensordict.nn import TensorDictModule as Mod, TensorDictSequential as Seq

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=

## Designing a policy

The next step is to build our policy.
We'll be making a regular, deterministic
version of the actor to be used within the
`loss module <gs_optim>` and during
`evaluation <gs_logging>`.
Next, we will augment it with an exploration module
for `inference <gs_storage>`.



In [3]:
from torchrl.modules import EGreedyModule, MLP, QValueModule

value_mlp = MLP(out_features=env.action_spec.shape[-1], num_cells=[64, 64])
value_net = Mod(value_mlp, in_keys=["observation"], out_keys=["action_value"])
policy = Seq(value_net, QValueModule(spec=env.action_spec))
exploration_module = EGreedyModule(
    env.action_spec, annealing_num_steps=100_000, eps_init=0.5
)
policy_explore = Seq(policy, exploration_module)

## Data Collector and replay buffer

Here comes the data part: we need a
`data collector <gs_storage_collector>` to easily get batches of data
and a `replay buffer <gs_storage_rb>` to store that data for training.




In [4]:
from torchrl.collectors import SyncDataCollector
from torchrl.data import LazyTensorStorage, ReplayBuffer

init_rand_steps = 5000
frames_per_batch = 100
optim_steps = 10
collector = SyncDataCollector(
    env,
    policy_explore,
    frames_per_batch=frames_per_batch,
    total_frames=-1,
    init_random_frames=init_rand_steps,
)
rb = ReplayBuffer(storage=LazyTensorStorage(100_000))

from torch.optim import Adam

## Loss module and optimizer

We build our loss as indicated in the `dedicated tutorial <gs_optim>`, with
its optimizer and target parameter updater:



In [5]:
from torchrl.objectives import DQNLoss, SoftUpdate

loss = DQNLoss(value_network=policy, action_space=env.action_spec, delay_value=True)
optim = Adam(loss.parameters(), lr=0.02)
updater = SoftUpdate(loss, eps=0.99)

## Logger

We'll be using a CSV logger to log our results, and save rendered videos.




In [6]:
from torchrl._utils import logger as torchrl_logger
from torchrl.record import CSVLogger, VideoRecorder

path = "./training_loop"
logger = CSVLogger(exp_name="dqn", log_dir=path, video_format="mp4")
video_recorder = VideoRecorder(logger, tag="video")
record_env = TransformedEnv(
    GymEnv("CartPole-v1", from_pixels=True, pixels_only=False), video_recorder
)

/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

## Training loop

Instead of fixing a specific number of iterations to run, we will keep on
training the network until it reaches a certain performance (arbitrarily
defined as 200 steps in the environment -- with CartPole, success is defined
as having longer trajectories).




In [7]:
total_count = 0
total_episodes = 0
t0 = time.time()
for i, data in enumerate(collector):
    # Write data in replay buffer
    rb.extend(data)
    max_length = rb[:]["next", "step_count"].max()
    if len(rb) > init_rand_steps:
        # Optim loop (we do several optim steps
        # per batch collected for efficiency)
        for _ in range(optim_steps):
            sample = rb.sample(128)
            loss_vals = loss(sample)
            loss_vals["loss"].backward()
            optim.step()
            optim.zero_grad()
            # Update exploration factor
            exploration_module.step(data.numel())
            # Update target params
            updater.step()
            if i % 10:
                torchrl_logger.info(f"Max num steps: {max_length}, rb length {len(rb)}")
            total_count += data.numel()
            total_episodes += data["next", "done"].sum()
    if max_length > 200:
        break

t1 = time.time()

torchrl_logger.info(
    f"solved after {total_count} steps, {total_episodes} episodes and in {t1-t0}s."
)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

2025-08-31 02:06:05,115 [torchrl][INFO]    Max num steps: 100, rb length 5200 [END]
2025-08-31 02:06:05,125 [torchrl][INFO]    Max num steps: 100, rb length 5200 [END]
2025-08-31 02:06:05,137 [torchrl][INFO]    Max num steps: 100, rb length 5200 [END]
2025-08-31 02:06:05,148 [torchrl][INFO]    Max num steps: 100, rb length 5200 [END]
2025-08-31 02:06:05,160 [torchrl][INFO]    Max num steps: 100, rb length 5200 [END]
2025-08-31 02:06:05,171 [torchrl][INFO]    Max num steps: 100, rb length 5200 [END]
2025-08-31 02:06:05,181 [torchrl][INFO]    Max num steps: 100, rb length 5200 [END]
2025-08-31 02:06:05,192 [torchrl][INFO]    Max num steps: 100, rb length 5200 [END]
2025-08-31 02:06:05,202 [torchrl][INFO]    Max num steps: 100, rb length 5200 [END]
2025-08-31 02:06:05,212 [torchrl][INFO]    Max num steps: 100, rb length 5200 [END]
2025-08-31 02:06:05,387 [torchrl][INFO]    Max num steps: 100, rb length 5300 [END]
2025-08-31 02:06:05,399 [torchrl][INFO]    Max num steps: 100, rb length 530

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


2025-08-31 02:06:05,799 [torchrl][INFO]    Max num steps: 100, rb length 5400 [END]
2025-08-31 02:06:05,809 [torchrl][INFO]    Max num steps: 100, rb length 5400 [END]
2025-08-31 02:06:05,820 [torchrl][INFO]    Max num steps: 100, rb length 5400 [END]
2025-08-31 02:06:06,005 [torchrl][INFO]    Max num steps: 100, rb length 5500 [END]
2025-08-31 02:06:06,017 [torchrl][INFO]    Max num steps: 100, rb length 5500 [END]
2025-08-31 02:06:06,029 [torchrl][INFO]    Max num steps: 100, rb length 5500 [END]
2025-08-31 02:06:06,041 [torchrl][INFO]    Max num steps: 100, rb length 5500 [END]
2025-08-31 02:06:06,054 [torchrl][INFO]    Max num steps: 100, rb length 5500 [END]
2025-08-31 02:06:06,068 [torchrl][INFO]    Max num steps: 100, rb length 5500 [END]
2025-08-31 02:06:06,082 [torchrl][INFO]    Max num steps: 100, rb length 5500 [END]
2025-08-31 02:06:06,096 [torchrl][INFO]    Max num steps: 100, rb length 5500 [END]
2025-08-31 02:06:06,109 [torchrl][INFO]    Max num steps: 100, rb length 550

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


2025-08-31 02:06:07,231 [torchrl][INFO]    Max num steps: 210, rb length 5900 [END]
2025-08-31 02:06:07,241 [torchrl][INFO]    Max num steps: 210, rb length 5900 [END]
2025-08-31 02:06:07,252 [torchrl][INFO]    Max num steps: 210, rb length 5900 [END]
2025-08-31 02:06:07,262 [torchrl][INFO]    Max num steps: 210, rb length 5900 [END]
2025-08-31 02:06:07,273 [torchrl][INFO]    Max num steps: 210, rb length 5900 [END]
2025-08-31 02:06:07,284 [torchrl][INFO]    Max num steps: 210, rb length 5900 [END]
2025-08-31 02:06:07,295 [torchrl][INFO]    Max num steps: 210, rb length 5900 [END]
2025-08-31 02:06:07,304 [torchrl][INFO]    Max num steps: 210, rb length 5900 [END]
2025-08-31 02:06:07,307 [torchrl][INFO]    solved after 9000 steps, 400 episodes and in 9.475800514221191s. [END]


## Rendering

Finally, we run the environment for as many steps as we can and save the
video locally (notice that we are not exploring).



In [17]:
record_env.rollout(max_steps=1000, policy=policy)
video_recorder.dump()

/usr/local/lib/python3.12/dist-packages/torchvision/io/_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(


ImportError: PyAV is not installed, and is necessary for the video operations in torchvision.
See https://github.com/mikeboers/PyAV#installation for instructions on how to
install PyAV on your system.


This is what your rendered CartPole video will look like after a full
training loop:

.. figure:: /_static/img/cartpole.gif

This concludes our series of "Getting started with TorchRL" tutorials!
Feel free to share feedback about it on GitHub.


